# SQLAlchemy ORM with French Cities & Departments

This notebook demonstrates SQLAlchemy's Object-Relational Mapping (ORM) using a real dataset of French cities and departments.

## Database Connection Setup

We connect to a SQLite database containing French geographical data:
- **SQLite**: File-based database, perfect for learning and prototyping
- **echo=True**: Shows all SQL queries SQLAlchemy generates (educational!)
- **Engine**: SQLAlchemy's "factory" for database connections


In [11]:
from sqlalchemy import create_engine
from pathlib import Path


p_departement = Path.cwd() / "data" / "villes_france.db"
engine = create_engine(f"sqlite:///{p_departement}")

engine

Engine(sqlite:////home/kevin-work/Documents/Seafile/COURS/Advanced_Programming/code-examples/04-class_for_sql/data/villes_france.db)

## ORM Class Definitions

**Object-Relational Mapping (ORM)** lets us work with database tables as Python classes:

### Key Concepts:
- **DeclarativeBase**: All ORM classes inherit from this
- **Mapped[type]**: Type hints that SQLAlchemy understands
- **mapped_column()**: Defines database column properties
- **ForeignKey**: Creates relationships between tables

### Table Relationships:
- `Ville.department` → `Departement.departement_code` (many-to-one)
- Each city belongs to exactly one department
- Each department can have many cities


In [12]:
# SQLAlchemy ORM Imports - Essential building blocks
from sqlalchemy import ForeignKey      # Creates relationships between tables
from sqlalchemy import String, Text    # Column data types (String = VARCHAR, Text = unlimited text)
from sqlalchemy.orm import DeclarativeBase  # Base class for all ORM models
from sqlalchemy.orm import Mapped       # Type annotation that tells SQLAlchemy about column types
from sqlalchemy.orm import mapped_column # Defines actual database columns with constraints
from sqlalchemy.orm import relationship  # Defines object-level relationships (not used here but imported)

# Base class - ALL your ORM classes must inherit from this
# Think of it as the "blueprint maker" for database tables
class Base(DeclarativeBase):
    pass

# Departement class = "departement" table in database
class Departement(Base):
    # __tablename__ tells SQLAlchemy which database table this class represents
    __tablename__ = "departement"

    # Column definitions using modern SQLAlchemy 2.0 syntax:
    # Mapped[type] = type hint for Python + SQLAlchemy
    # mapped_column() = actual database column configuration

    # Primary key: unique identifier for each row
    # SQLAlchemy automatically makes this AUTOINCREMENT in SQLite
    departement_id: Mapped[int] = mapped_column(primary_key=True)

    # Text columns: unlimited length strings
    # In other databases, Text = CLOB/LONGTEXT, String(50) = VARCHAR(50)
    departement_code: Mapped[str] = mapped_column(Text)  # e.g., "75", "971"
    departement_nom: Mapped[str] = mapped_column(Text)   # e.g., "Paris", "Guadeloupe"

    def is_outremer(self) -> bool:
        """
        Custom Python method - this is NOT a database column!
        Business logic: French overseas departments have 3-digit codes (971, 972, etc.)
        Metropolitan departments have 2-digit codes (01, 02, ..., 95)
        """
        return len(self.departement_code) == 3

# Ville class = "villes" table in database
class Ville(Base):
    __tablename__ = "villes"
    
    # Primary key column - unique ID for each city
    id: Mapped[int] = mapped_column(primary_key=True)
    # Note: AUTOINCREMENT is automatic for integer primary keys in SQLAlchemy

    # FOREIGN KEY RELATIONSHIP - This is how tables connect!
    # This column stores department codes and references departement.departement_code
    # Creates SQL: FOREIGN KEY (department) REFERENCES departement(departement_code)
    department: Mapped[str] = mapped_column(ForeignKey("departement.departement_code"))

    # Regular data columns with different types:
    name: Mapped[str] = mapped_column(Text)              # City name (unlimited text)
    population_2012: Mapped[int] = mapped_column()       # Integer (SQL: INTEGER)
    surface: Mapped[float] = mapped_column()             # Decimal number (SQL: REAL)

    # Unique constraint: no two cities can have the same commune code
    # Creates SQL: UNIQUE(commune_code)
    commune_code: Mapped[str] = mapped_column(Text, unique=True)
    
    def __repr__(self) -> str:
        """
        String representation for debugging/printing
        When you print(ville_object), this is what you'll see
        Very useful for debugging queries!
        """
        return f"Ville(name={self.name}, department={self.department}, pop={self.population_2012})"



In [16]:
from sqlalchemy.orm import Session
from sqlalchemy import select

session = Session(engine)



The **Session** is your main interface to the database:

- **Tracks objects**: Knows which objects are new, modified, or deleted
- **Manages transactions**: Groups operations together
- **Identity map**: Ensures one Python object per database row
- **Lazy loading**: Can fetch related data automatically

Think of it as your "conversation" with the database.

In [25]:
stmt = (
    select(Departement)
    .where(Departement.departement_code.startswith("9")) # All the departemenens starting with 6
)
ans = session.execute(stmt) # Returns Result object
ans

This is **class-level attribute access**:

- `Departement.departement_code` → References the column definition in your class
- `.startswith("6")` → SQLAlchemy translates this to SQL: `WHERE departement_code LIKE '6%'`
- **No SQL strings needed!** Pure Python expressions

In [24]:

for dept in ans.scalars():
    # dept is now a full Python object with all your class methods/attributes
    print(dept.departement_nom, dept.departement_code) # Access database columns, As Python attributes


In [26]:
ans = session.execute(stmt) # Returns Result object
for dept in ans.scalars():
    # If you had methods in your class, you could call them here too!
    print(f"The department {dept.departement_nom} is {'not ' if not dept.is_outremer() else ''}overseas.")

The department Territoire de Belfort is not overseas.
The department Essonne is not overseas.
The department Hauts-de-Seine is not overseas.
The department Seine-Saint-Denis is not overseas.
The department Val-de-Marne is not overseas.
The department Val-d'oise is not overseas.
The department Mayotte is overseas.
The department Guadeloupe is overseas.
The department Guyane is overseas.
The department Martinique is overseas.
The department Réunion is overseas.


**What scalars() does:**

- **Input**: `Row(Departement(...))` - wrapped objects
- **Output**: `Departement(...)` - direct objects
- **Why**: Makes iteration cleaner and more Pythonic
The scalars() method unwraps these Row objects so you get your actual model objects directly:
Bottom line: scalars() removes the Row wrapper so you can work with your Python objects directly instead of having to use row[0] everywhere.

## Complex Query Example: Joins + Aggregations

Let's answer: **"Which overseas department has the most cities with population > 500?"**

### Query Building Blocks:

**1. SELECT clause:**
- `Departement` → Full department object  
- `func.count(Ville.id).label('total_cities')` → Count cities per department

**2. JOIN:**
- Links tables: `Ville.department == Departement.departement_code`
- Now we can filter/group across both tables

**3. WHERE clauses:**
- `Ville.population_2012 > 500` → Only substantial cities
- `Departement.departement_code.startswith("97")` → Only overseas (97x codes)

**4. Aggregation:**
- `group_by(Departement.departement_id)` → One result per department
- `func.count()` → Count cities within each group

**5. Ordering & Limiting:**
- `order_by(...desc())` → Highest count first
- `limit(1)` → Just the winner


In [30]:
from sqlalchemy import func
#Let's say we want to know which overseas departement
# has the most cities of more than 500 inhabitants.
# why not...

stmt = (
    select(
        Departement,
        func.count(Ville.id)
        )
    .join(Departement, Ville.department == Departement.departement_code)    
    .where(Ville.population_2012 > 500)
    .where(Departement.departement_code.startswith("97"))
    .group_by(Departement.departement_id)
    .order_by(func.count(Ville.id).desc())
    .limit(5)
    )

### Query Execution & Result Handling

When your SELECT includes multiple items, SQLAlchemy returns **tuples**:
- `(Departement_object, count_value)`
- Use tuple unpacking: `dept, count = row`

**Two patterns:**
- `fetchall()` → Returns list of all rows (good for iteration)
- `fetchone()` → Returns just the first row (good for single results)


In [35]:
results = session.execute(stmt).fetchall()

In [37]:
for row in results:
    dept, count = row  # Unpack the tuple
    print(f"Department: {dept.departement_nom}, Cities > 500: {count}")



Department: Guadeloupe, Cities > 500: 34
Department: Martinique, Cities > 500: 34
Department: Réunion, Cities > 500: 24
Department: Guyane, Cities > 500: 20
Department: Mayotte, Cities > 500: 17


In [38]:
# Or just the first result since we know we do limit 1:
dept, count = session.execute(stmt).fetchone()
print(f"Winner: {dept.departement_nom} with {count} cities")

Winner: Guadeloupe with 34 cities
